In [9]:
from IPython.display import HTML

HTML('''<script>
            code_show=true; 
            function code_toggle() {
                if (code_show){
                    $('div.input').hide();
                } else {
                    $('div.input').show();
                }
                    code_show = !code_show
            } 
            $( document ).ready(code_toggle);
        </script>
        <form action="javascript:code_toggle()"><input type="submit" value="View/Hide Raw Code"></form>''')

# Converting the GenBank File to SBOL
1. Upload the GenBank file or place in the same folder.
2. Enter the file name and the URI prefix (something like https://bu.edu/ben) that you want to use.
3. Enter a version (optional).
4. Hit send and it will create an SBOL file with the same name.

In [10]:
from sbol import *
import ipywidgets as widgets
from IPython.display import Javascript, display, clear_output
from ipyupload import FileUpload
import os
import sys
CURRENTDIR = os.getcwd()
SRCDIR = os.path.join(CURRENTDIR, 'src')
if sys.path[0] != SRCDIR:
    sys.path.insert(0, SRCDIR)
import convert

# Widgets
upload = FileUpload(
    accept='.gb',
    multiple=True,
    disabled=False,
    style_button='color: white; background-color: teal',
    compress_level=0
)
URIPrefix = widgets.Text(
    value='', 
    placeholder='Enter URI Prefix',
    description='URL:'
)
version = widgets.Text(
    placeholder='Enter Version (optional)',
    description='Version:'
)
sendButton = widgets.Button(
    description='Send',
    disabled=True,
    tooltip='Converts Genbank file to SBOL'
)
out = widgets.Output()

# Displaying Widgets
display(upload)
display(URIPrefix)
display(version)
display(sendButton)
display(out)

# Global Variables for Whole Notebook
doc = Document()
Config.setOption('sbol_compliant_uris', True)
Config.setOption('sbol_typed_uris', False)

originalCDs = []
addedCDs = []
cdDictionary = {}

# Listener to enable the Send button
def checkInputs(sender):
    if URIPrefix.value == '' or len(upload.value) == 0:
        sendButton.disabled = True
    else:
        sendButton.disabled = False

# Listener for the Send button push -- adds GenBank files to Document
@out.capture()
def sendRequest(sender):
    global originalCDs, cdDictionary, doc
    clear_output()
        
    doc = Document()
    setHomespace(URIPrefix.value)

    cdDicitonary = {}
    originalCDs = []
    addedCDs = []
    
    try:
        convert.getSBOLFiles(doc, originalCDs, cdDictionary, upload.value, URIPrefix.value, version.value)
    except:
        print('You need to input GenBank files.')
        return
    
    print('Success!')
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

# Connecting Observers
upload.observe(checkInputs)
URIPrefix.observe(checkInputs)
sendButton.on_click(sendRequest)


FileUpload(accept='.gb', multiple=True, style_button='color: white; background-color: teal')

Text(value='', description='URL:', placeholder='Enter URI Prefix')

Text(value='', description='Version:', placeholder='Enter Version (optional)')

Button(description='Send', disabled=True, style=ButtonStyle(), tooltip='Converts Genbank file to SBOL')

Output()

# Add a Component
Enter the name of the component, the type, and the role (optional).

In [17]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown
import os
import sys
CURRENTDIR = os.getcwd()
SRCDIR = os.path.join(CURRENTDIR, 'src')
if sys.path[0] != SRCDIR:
    sys.path.insert(0, SRCDIR)
from componentsingle import componentTypes, componentRoles, typesMap, rolesMap, addComponent


# Widgets
componentName = widgets.Text(
    value='',
    placeholder='Enter a component name',
    description='Name:'
)
typeDrop = widgets.Dropdown(
    options=componentTypes,
    description='Type:'
)
roleDrop = widgets.Dropdown(
    options=componentRoles,
    description='Role:'
)
createButton = widgets.Button(
    description='Create',
    disabled=True,
    tooltip='Creates a new Component Definition'
)
outBox = widgets.Output()

# Displaying Widgets
display(componentName)
display(typeDrop)
display(roleDrop)
display(createButton)
display(outBox)

# Listener to enable Create button
def checkText(sender):
    if componentName.value == '':
        createButton.disabled = True
    else:
        createButton.disabled = False

# Listener for Create button push -- creates and adds Component Definition to Document
@outBox.capture()
def createButtonHandler(sender):
    global doc, addedCDs
    clear_output()
    
    addComponent(doc, addedCDs, componentName.value, typeDrop.value, roleDrop.value)
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

componentName.observe(checkText)
createButton.on_click(createButtonHandler)

Text(value='', description='Name:', placeholder='Enter a component name')

Dropdown(description='Type:', options=('DNA', 'RNA', 'Protein', 'Small Molecule', 'Complex'), value='DNA')

Dropdown(description='Role:', options=('', 'Miscellaneous', 'Promoter', 'RBS', 'CDS', 'Terminator', 'Gene', 'O…

Button(description='Create', disabled=True, style=ButtonStyle(), tooltip='Creates a new Component Definition')

Output()

# Component Definitions

In [18]:
from sbol import *
import ipywidgets as widgets
from componentdefinitions import fixAndCreateWidgets

mainBox = widgets.VBox(children=fixAndCreateWidgets(doc, originalCDs, addedCDs, cdDictionary))

display(mainBox)

# Add Components to Modules


In [19]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown
from componentbuckets import *

# Nested Modules
device_test_context = ModuleDefinition('context')
device_test = ModuleDefinition('device_test')
device = ModuleDefinition('device')

device_test_submod = device_test_context.modules.create('device_test')
device_test_submod.definition = device_test.identity

device_submod = device_test.modules.create('device')
device_submod.definition = device.identity


# Get modules that are in the Document        
found_device_test_context = None
found_device_test = None
found_device = None

for md in doc.moduleDefinitions:
    if md.identity == device_test_context.identity:
        found_device_test_context = md
    if md.identity == device_test.identity:
        found_device_test = md
    if md.identity == device.identity:
        found_device = md

if found_device_test_context != None:
    device_test_context = found_device_test_context
    device_test = found_device_test
    device = found_device
else:
    doc.addModuleDefinition(device)
    doc.addModuleDefinition(device_test)
    doc.addModuleDefinition(device_test_context)

# Widgets
leftRightButton = widgets.Button(description='>>', layout=widgets.Layout(width='50px')) # noqa
leftLeftButton = widgets.Button(description='<<', layout=widgets.Layout(width='50px')) # noqa
rightRightButton = widgets.Button(description='>>', layout=widgets.Layout(width='50px')) # noqa
rightLeftButton = widgets.Button(description='<<', layout=widgets.Layout(width='50px')) # noqa

leftVBox = widgets.VBox()
rightVBox = widgets.VBox()
leftVBox.children = [leftRightButton, leftLeftButton]
rightVBox.children = [rightRightButton, rightLeftButton]

horizontalBox = widgets.HBox(layout = widgets.Layout(align_items = 'center', justify_content = 'space-around'))


moduleNames = ['Device-Test-Context', 'Device-Test', 'Device']
modulesDictionary = {'Device-Test-Context': device_test_context,
                     'Device-Test': device_test,
                     'Device': device}

# Text widgets for device-test-context, device-test, and device names
nameList = []
# Select Multiple widgets for device-test-context, device-test, and device
selectedLists = []

componentLists = {}  # module name : list of components
cdDisplayIDMap = {}  # cd display id : component definition

componentLists['Device-Test-Context'] = createDTCList(doc, originalCDs, addedCDs, cdDisplayIDMap)
componentLists['Device-Test'] = []
componentLists['Device'] = []

fcDictionary = {} # name w/ plasmid name : name w/o plasmid name


horizontalBox.children = createColumnWidgets(moduleNames, componentLists, nameList, selectedLists, leftVBox, rightVBox)
display(horizontalBox)

# Device-Test-Context -> Device-Test
def leftRightClickResponder(sender):
    leftRightClick(selectedLists, componentLists)

# Device-Test -> Device-Test-Context
def leftLeftClickResponder(sender):
    leftLeftClick(selectedLists, componentLists)

# Device-Test -> Device
def rightRightClickResponder(sender):
    rightRightClick(selectedLists, componentLists)

# Device -> Device-Test
def rightLeftClickResponder(sender):
    rightLeftClick(selectedLists, componentLists)

leftRightButton.on_click(leftRightClickResponder)
leftLeftButton.on_click(leftLeftClickResponder)
rightRightButton.on_click(rightRightClickResponder)
rightLeftButton.on_click(rightLeftClickResponder)


finalizeModulesButton = widgets.Button(description = 'Finalize', tooltip = 'Finalize modules', disabled = True)
display(finalizeModulesButton)

displayMessages = widgets.Output()
display(displayMessages)


@displayMessages.capture()
def printMessage(message):
    clear_output()
    print(message)

def observer(sender):
    for nameWidget in nameList:
        if nameWidget.value == '':
            finalizeModulesButton.disabled = True
            return
    
    finalizeModulesButton.disabled = False


plasmidPartDictionary = {}
addedPlasmidParts = {}


def finalize(sender):
    global device_test_context, device_test, device

    resetModules(device_test_context, device_test, device)
    setNames(device_test_context, device_test, device, nameList)
    createFunctionalComponents(doc, selectedLists, cdDisplayIDMap, modulesDictionary, moduleNames, fcDictionary)
    addPlasmidParts(doc, originalCDs, device_test, device, plasmidPartDictionary, addedPlasmidParts)
    
    printMessage('Success!')
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    

for textWidget in nameList:
    textWidget.observe(observer)


finalizeModulesButton.on_click(finalize)


Button(description='Finalize', disabled=True, style=ButtonStyle(), tooltip='Finalize modules')

Output()

<IPython.core.display.Javascript object>

# Adding Interactions to the Modules

In [24]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown
from addinginteractions import *

module_names = [device_test_context.name, device_test.name, device.name]
namesDictionary = {}
customNameDictionary = {device_test_context.name: device_test_context,
                        device_test.name: device_test,
                        device.name: device}  # custom name : moduledefinition

    
moduleToAnnotate = widgets.Dropdown(
    options=module_names,
    description='Module:'
)

enterButton = widgets.Button(
    description='Enter',
    tooltip='Pick module'
)

interactionName = widgets.Text(
    placeholder='Enter interaction name',
    description='Name:',
    style={'description_width': '125px'},
    layout=widgets.Layout(width='450px')
)

interactionSelect = widgets.Dropdown(
    options=typesInteractions,
    description='Interaction:',
    style={'description_width': '125px'},
    layout=widgets.Layout(width='325px')
)

horizontalBox = widgets.HBox()  # for showing the components in each of the plasmids

addInteraction = widgets.Button(
    description='Add',
    tooltip='Add interaction'
)

doneButton = widgets.Button(
    description='Done',
    tooltip='Finish adding interactions'
)

finalizeInteraction = widgets.Button(
    description='Finalize',
    tooltip='Finalize interaction'
)

messageBox = widgets.Output()

interactionWidgets = widgets.VBox()
interactionChildren = []

participationWidgets = widgets.VBox()
participationChildren = []

display(moduleToAnnotate)
display(enterButton)
display(interactionWidgets)
display(messageBox)

@messageBox.capture()
def printMessages(message):
    clear_output()
    print(message)

def on_select(sender):    
    global interactionChildren
    
    interactionName.value = ''
    interactionSelect.value = None
    
    interactionChildren = []
    interactionWidgets.children = interactionChildren   
    
    horizontalBox.children = createHBoxChildren(moduleToAnnotate.value, customNameDictionary, fcDictionary)
    
    interactionChildren.append(widgets.HTML('<br>'))
    interactionChildren.append(interactionName)
    interactionChildren.append(interactionSelect)
    interactionChildren.append(widgets.HTML('<br>'))
    interactionChildren.append(horizontalBox)
    interactionChildren.append(widgets.HTML('<br>'))
    interactionChildren.append(addInteraction)
    
    interactionWidgets.children = interactionChildren
    
participantDictionary = {} # fc display id : participation
    
def addInteractionResponder(sender):
    if interactionSelect.value is None:
        printMessages('You need an interaction type!')
        return
    printMessages('')
    
    participationChildren, selectedFC = createParticipationChildren(horizontalBox.children)
    
    try:
        createInteraction(customNameDictionary,
                      moduleToAnnotate.value,
                      interactionName.value,
                      interactionSelect.value,
                      participantDictionary,
                      selectedFC)
    except:
        printMessages(interactionName.value + ' already exists!')
        return

   
    participationWidgets.children = participationChildren
    
    interactionChildren.append(widgets.HTML('<br>'))
    interactionChildren.append(participationWidgets)
    interactionChildren.append(finalizeInteraction)
    interactionWidgets.children = interactionChildren
    
            

def finalizeResponder(sender):
    global interactionChildren, participationChildren
    createMapsTos(doc, participationWidgets.children, participantDictionary, plasmidPartDictionary, addedPlasmidParts)
                
    interactionChildren = []
    participationChildren = []
    
    interactionWidgets.children = interactionChildren
    participationWidgets.children = participationChildren
    
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    printMessages('Success!')


enterButton.on_click(on_select)
addInteraction.on_click(addInteractionResponder)
finalizeInteraction.on_click(finalizeResponder)


Dropdown(description='Module:', options=('dtc', 'dt', 'd'), value='dtc')

Button(description='Enter', style=ButtonStyle(), tooltip='Pick module')

VBox()

Output()

# Modules

In [25]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown
from displaymoduledefinitions import createParentAccordion

modules = [device_test_context, device_test, device]
accordion = createParentAccordion(modules, modulesDictionary)

display(accordion)


Accordion(children=(VBox(children=(HTML(value='<b><h5>(Device-Test-Context)</h5></b><u><dt>Modules</dt></u><dd…

# Upload to SynBioHub
To upload to SynBioHub:
1. Enter your username and password (you need to have an account to upload).
2. Enter a name, description, and version for the Collection.
3. Press Submit.

In [26]:
from sbol import *
import ipywidgets as widgets
from IPython.display import clear_output, Markdown

loginLabel = widgets.HTML('<font size="4"><b>Login to SynBioHub</b></font>')

usernameField = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter email',
    description = 'Email:',
    disabled = False,
)

passwordField = widgets.Password(description = 'Password:', placeholder = 'Enter password')

loginButton = widgets.Button(
    description = 'Login',
    disabled = False,
    button_style = '',
    tooltip = 'Login to SynBioHub',
    icon = ''
)

loginMessageBox = widgets.Output()

submitButton = widgets.Button(
    description = 'Submit',
    disabled = False,
    button_style = '',
    tooltip = 'Submit annotated Document',
    icon = ''
)

errorBox = widgets.Output()

# Name = Display Id
propertiesLabel = widgets.HTML('<font size="4"><b>Enter Name, Description, and Version</b></font>')

docName = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter name',
    description = 'Name:',
    disabled = False,
)

docDescription = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter description',
    description = 'Description:',
    disabled = False,
)

docVersion = widgets.componentName = widgets.Text(
    value = '',
    placeholder = 'Enter version',
    description = 'Version:',
    disabled = False,
)

display(loginLabel)
display(usernameField)
display(passwordField)
display(loginButton)
display(loginMessageBox)
display(propertiesLabel)
display(docName)
display(docDescription)
display(docVersion)
display(submitButton)
display(errorBox)

sbh = PartShop('https://synbiohub.org')

@loginMessageBox.capture()
def loginSBH(sender):
    clear_output()
    if usernameField.value == '':
        print('You need a username!')
        return
    if passwordField.value == '':
        print('You need a password!')
        return
    
    sbh.login(usernameField.value, passwordField.value)
    message = sbh.submit(Document())
    
    if message == 'Login required':
        print('Incorrect login!')
    else:
        print('Logged in!')

@errorBox.capture()
def submitSBH(sender):
    clear_output()
    if docName.value == '':
        print('You need a name for the Collection!')
        return
    if docDescription.value == '':
        print('You need a description for the Collection!')
        return
    if docVersion.value == '':
        print('You need a version for this Collection!')
        return
    
    doc.displayId = docName.value
    doc.name = docName.value
    doc.description = docDescription.value
    doc.version = docVersion.value

    message = sbh.submit(doc)
    
    print(message)

loginButton.on_click(loginSBH)
submitButton.on_click(submitSBH)

HTML(value='<font size="4"><b>Login to SynBioHub</b></font>')

Text(value='', description='Email:', placeholder='Enter email')

Password(description='Password:', placeholder='Enter password')

Button(description='Login', style=ButtonStyle(), tooltip='Login to SynBioHub')

Output()

...

Text(value='', description='Name:', placeholder='Enter name')

Text(value='', description='Description:', placeholder='Enter description')

Text(value='', description='Version:', placeholder='Enter version')

Button(description='Submit', style=ButtonStyle(), tooltip='Submit annotated Document')

Output()

In [31]:
# THIS CODE BLOCK IS JUST TO TEST RANDOM VARIABLES AND SUCH


# for i in device_test_context.functionalComponents:
#     print(i.displayId)
    
# doc.write('whatisthat.xml')
# print(doc)

# customNameDictionary

# device.functionalComponents.clear()
# device_test.functionalComponents.clear()
# device_test_context.functionalComponents.clear()
# for i in participationWidgets.children:
#     print(i.value)

# dtcListWidget = selectedLists[0]
# for i in dtcListWidget.options:
#     print(i)
# for i in device.functionalComponents:
#     print(i)
    
# for i in selectedLists[2].options:
#     print(i)

# print(cdDisplayIDMap)

# print(participantDictionary)
# foo
# doc.write('hi.xml')
cd = doc.find('https://bu.edu/ben/BBa_K808000__CDS__misc_feature0__misc_feature1__promoter/1.0.0').cast(ComponentDefinition)
for i in cd.sequenceAnnotations:
    print(len(i.locations))
# #     if '2' in i.displayId:
# #         for j in i.locations:
# #             j = j.cast(Range)
# print(len(cd.sequenceAnnotations))
# for i in cd.sequenceAnnotations:
#     for j in i.locations:
#         print(j.name)
            
# doc.write('hi2.xml')






1
1
1
1


'Invalid. sbol-10902:\x00 Strong Validation Error:\x00 The locations property of a SequenceAnnotation is REQUIRED and MUST contain a non-empty set of Location objects.  \x00Reference: SBOL Version 2.3.0 Section 7.7.4 on page 32 :\x00 https://bu.edu/ben/BBa_K808000__CDS__misc_feature0__misc_feature1__promoter/BBa_K808000_annotation0/1.0.0\x00  sbol-12303:\x00 Strong Validation Error:\x00 The rdfType property of a GenericTopLevel object MUST NOT contain a QName in the “http://sbols.org/v2#” namespace. \x00Reference: SBOL Version 2.3.0 Section 7.14.2 on page 55 :\x00 https://bu.edu/ben/BBa_K808000__CDS__misc_feature0__misc_feature1__promoter/BBa_K808000_annotation0/testing/1.0.0\x00  sbol-12303:\x00 Strong Validation Error:\x00 The rdfType property of a GenericTopLevel object MUST NOT contain a QName in the “http://sbols.org/v2#” namespace. \x00Reference: SBOL Version 2.3.0 Section 7.14.2 on page 55 :\x00 https://bu.edu/ben/BBa_K808000__CDS__misc_feature0__misc_feature1__promoter/BBa_K808